## Environment setup

### Library preparation

In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:
import time, os, io
from PIL import ImageGrab
from IPython.display import Image, display
import clipboard
from scipy import stats
from si_prefix import si_format
from tqdm import tqdm, tqdm_notebook

import pandas as pd
import numpy as np
from io import StringIO 
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
plt.close('all')

import sys 
sys.path.append('..')
from meas_util import *

import qcodes as qc
from qcodes import Station, load_by_run_spec, load_by_guid
from qcodes.instrument.base import Instrument
from qcodes.dataset.experiment_container import (Experiment,
                                                 load_last_experiment,
                                                 new_experiment)
from qcodes.dataset.database import initialise_database
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.plotting import plot_by_id, get_data_by_id, plot_dataset
from qcodes.dataset.data_set import load_by_id
# from qcodes.dataset.data_export import get_shaped_data_by_runidb

C:\Users\KelvinOX25\Anaconda3\lib\site-packages\qcodes\dataset\database.py:13: UserWarning: The module `qcodes.dataset.database` is deprecated.
Public features are available at the import of `qcodes`.
Private features are available in `qcodes.dataset.sqlite.*` modules.
  warnings.warn('The module `qcodes.dataset.database` is deprecated.\n'


In [3]:
from qcodes.instrument_drivers.Lakeshore.Model_372 import Model_372
from qcodes.instrument_drivers.tektronix.AWG3252_Isrc import AWG3252_Isrc
from qcodes.instrument_drivers.HP.HP34401 import HP34401
from qcodes.instrument_drivers.HP.HP34401_DL1201 import HP34401_DL1201
from qcodes.instrument_drivers.Keithley.Keithley_2600_channels import *
from qcodes.instrument_drivers.tektronix.AWG5204_Isrc import AWG5204_Isrc

from qcodes.instrument_drivers.Keysight.Keysight_B2962A_Isrc import B2962A_Isrc


from qcodes.instrument_drivers.Keithley.Keithley_6220 import Keithley_6220
from qcodes.instrument_drivers.Keithley.Keithley_6220_Isrc import Keithley_6220_Isrc

In [4]:
from JJmeasurement import *

C:\Users\KelvinOX25\Anaconda3\lib\site-packages\tqdm\autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


### Instrument nicknames and QC database initialization

In [5]:
try:
   Instrument.close_all()
except KeyError:
   pass    
except NameError:
   pass 

K2602 =  Keithley_2600(name = 'K2602', address = "GPIB::26::INSTR" )
Flux_chA = K2602.smua.curr
Flux_chB = K2602.smub.curr
Bfield =  Flux_chA
Bfield.step = 10e-6
Bfield.inter_delay = .1



# Isrc = AWG5204_Isrc('gen', 'TCPIP0::AWG52000-XXXX::inst0::INSTR', ch = 1, R_bias = 1e9, Attn = 1, timeout = 20)

# Isrc = B2962A_Isrc(name = 'B2962A', address = "GPIB::23::INSTR" )

# Isrc = Keithley_6220(name = 'K6220', address = "GPIB::13::INSTR" )
# Isrc.set_R_Attn ( R_bias = 1, Attn = 1 )




# Isrc = AWG3252_Isrc('gen', 'TCPIP0::192.168.13.32::inst0::INSTR',  R_bias = 1e9, Attn = 1)
# Isrc.init()

Vmeter = HP34401_DL1201('meter', 'GPIB0::8::INSTR', Gain = 100)
Vmeter.init('fast 6')


Isrc = Keithley_6220_Isrc(name = 'K6220', address = "GPIB::13::INSTR",
                          Rsh = 1e6, Rb = 1e9, Vmeter = Vmeter)




station = qc.Station(Isrc, Vmeter)

print ('Instrument initialization completed.')

Connected to: Keithley Instruments Inc. 2602A (serial:1272049, firmware:2.1.5) in 0.30s


[K6220(Keithley_6220_Isrc)] Snapshot: Could not update parameter: Irange


Instrument initialization completed.


In [6]:

LS370 = Model_372(name = 'LS370 input Ch', address = 'GPIB::12::INSTR', terminator='\n')
T8 = LS370.ch08.temperature

htr = LS370.sample_heater
htr_setpt = htr.setpoint

htr_setpt.set(0.030)
htr.P.set(30)
htr.I.set(5)
htr.D.set(1)
# PIDset(30,5,1)
htr.output_range('3.16mA')

Connected to: LSCI 370 (serial:370665, firmware:04102008) in 0.09s


### Database ini

In [7]:
SAMPLE = '59BBNX'
qc.config["core"]["db_location"] = r'\\JOSH-PC\OurData_OneDrive\Experiments_{}.db'.format(SAMPLE)

### Class JJmeas

In [8]:
tools = {'I' : Isrc.I,
         'V' : Vmeter.V,
         'B' : Flux_chA,
         'T': T8,
         'htr' : htr_setpt}

In [9]:
def setup():
    Isrc.I.set(0)

def cleanup():
    Isrc.I.set(0) 
    Bfield.set(0)



In [10]:
jj = JJmeas(sample = '59BBN1', tools = tools)
jj.setup = setup
jj.cleanup = cleanup

In [11]:
jj.tool_status(['B', 'T'])

{'B': '-1.7 µA', 'T': '39.0 mK'}

## Measurement

#### Pin assignment: 4ba,3ab

#### Constants

In [35]:
ZF   = 0.00e-3
FF   = 1.85e-3
#dF   = 0.45e-3
midF = 0.90e-3

#### Iscr choise

In [12]:
Isrc.ask_raw('SOUR:CURR:RANG?')

'2.000000E-07\n'

In [13]:
Isrc.write_raw('SOUR:CURR:RANG 1e-8')
Isrc.I.set(0.0e-9)

In [199]:
Isrc.close()
Isrc = Keithley_6220_Isrc(name = 'K6220', address = "GPIB::13::INSTR",
                          Rsh = 1e6, Rb = 1e9, Vmeter = Vmeter)

jj.tools['I'] =  Isrc.I


In [201]:
Isrc.I.get()

-6.185797122877277

In [203]:
Ke = Keithley_6220(name = 'K6220_bare', address = "GPIB::13::INSTR" )

In [204]:
Ke.I.get()

0.0

#### Vmeter regime

In [9]:
Vmeter.init('fast 6')

In [193]:
Isrc.I.set(-1.0e-12)
Vmeter.V.get()

0.00027847579

In [240]:
Vmeter.Voff 

0

### IVCs

In [21]:
Isrc.I.set(0e-12)

In [29]:
Bfield.set(0e-6)

In [13]:
jj.tool_status()

{'I': '-510.7 fA',
 'V': '242.2 µV',
 'B': '-1.8 µA',
 'T': '35.7 mK',
 'htr': '30.0 m'}

In [12]:
# N1: 4ba,3ab; NX: 1ab,2ab
SAMPLE = '59BBNX'
qc.config["core"]["db_location"] = r'\\JOSH-PC\OurData_OneDrive\Experiments_{}.db'.format(SAMPLE)
jj.IVC_udu (amp = 1e-12, stp = 0.01e-12, dt = .5, Ioff = -.35e-12, N_avg = 1, label = 'ZF_fin')

Starting experimental run with id: 936


An exception occured in measurement with guid: aaaaaaaa-0000-0000-0000-01714b44f85e;
Traceback:
Traceback (most recent call last):
  File "..\JJmeasurement.py", line 98, in IVC_cust
    time.sleep(dt)
KeyboardInterrupt



KeyboardInterrupt: 

In [23]:
Isrc.Irange.set(1e3*10e-9)
time.sleep(10)
jj.IVC_fwd (amp = 10e-9, stp = 0.1e-9, dt = .5, N_avg = 1, label = 'ZF_rough')

VisaIOError: ('VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.', "asking 'RDGK? 8\\n\\r' to <Model_372: LS370 input Ch>", 'getting LS370 input Ch_ch08_temperature')

In [15]:
Isrc.Irange.set(1e3*1e-12)
time.sleep(10)
Bfield.set(0e-3)
jj.IVC_udu (amp = 1e-12, stp = 0.01e-12, dt = .5, Ioff = -.35e-12, N_avg = 1, label = 'ZF_fin')
Bfield.set(1.85e-3)
jj.IVC_udu (amp = 1e-12, stp = 0.01e-12, dt = .5, Ioff = -.35e-12, N_avg = 1, label = 'FF_fin')
# jj.IVC_udu (amp = .7e-12, stp = .01e-12, dt = .5, N_avg = 10,  Ioff = -.2e-12)

Starting experimental run with id: 932


An exception occured in measurement with guid: aaaaaaaa-0000-0000-0000-01714b417ac6;
Traceback:
Traceback (most recent call last):
  File "..\JJmeasurement.py", line 98, in IVC_cust
    time.sleep(dt)
KeyboardInterrupt



KeyboardInterrupt: 

In [ ]:
i_list = udu_list(1e-12, 0.1e-12) -.25e-12

jj.IVC_cust (i_list, dt = .5, N_avg = 1)

### IVC B

#### New syntaxis to try

In [31]:
# B_list = udu_list ( 4e-3, 0.2e-3)
B_list = np.linspace(0, 1.85e-3,2)

[            jj.IVC_fwd (amp = 5e-12, stp = 0.01e-12, dt = .5, N_avg = 1)
for jj in    jj.Bscan( B_list = B_list )]


Bfield.set(0)


Starting experimental run with id: 897


Starting experimental run with id: 898

{ 'ids' : range(897,898+1), 'B' : np.linspace(0.00e+00,1.85e-03, 2), 'T' : 0.030, 'comm : '' }


In [174]:
# B_list = udu_list ( 4e-3, 0.2e-3)
B_list = np.linspace(0, 1.85e-3, 11)

i_list = udu_list(1e-12, 0.01e-12) -.25e-12



[            jj.IVC_cust (i_list, dt = .5, N_avg = 4)
for jj in    jj.Bscan( B_list = B_list )]

Bfield.set(0)


Starting experimental run with id: 44


Starting experimental run with id: 45


Starting experimental run with id: 46


Starting experimental run with id: 47


Starting experimental run with id: 48


...

Starting experimental run with id: 50


Starting experimental run with id: 51


Starting experimental run with id: 52


Starting experimental run with id: 53


Starting experimental run with id: 54



In [230]:
jj.ZF = 0
jj.FF = 1.85e-3
cos_list = [1,0]

[            jj.IVC_fwd (amp = 1e-12, stp = 0.1e-12, dt = .5, Ioff = -.35e-12)
for jj in    jj.Bscan( cos_list = cos_list )]

Starting experimental run with id: 63


Starting experimental run with id: 64

{ 'ids' : range(63,64+1), 'B' : np.linspace(0.00e+00,1.85e-03, 2), 'T' : 0.030, 'comm : '' }


[63, 64]

### T scan

In [210]:
5*11*6/60

5.5

In [135]:
htr.HTRset('3.16mA')

'Heater changed = 3.16mA'

In [150]:
htr.output_range('3.16mA')

In [109]:
T8.get()

0.0501275

In [ ]:
[            jj.IVC_fwd (amp = 1e-12, stp = 0.025e-12, dt = .5, N_avg = 100)
for jj in    jj.Tscan( [200e-3] )]

htr.set(0.030)

In [19]:
np.array( [50 + 25*i for i in range (11) ] )*1e-3

array([0.05 , 0.075, 0.1  , 0.125, 0.15 , 0.175, 0.2  , 0.225, 0.25 ,
       0.275])

In [21]:
htr_setpt.set(30e-3)

In [13]:
htr.output_range('3.16mA')

In [28]:
T_list = np.array( [50 + 25*i for i in range (11) ] )*1e-3
B_list = np.linspace(0, 1.85e-3, 11)

for jj in  jj.Tscan( T_list ):
    
    for jj in jj.Bscan(B_list):
    
        jj.IVC_udu (amp = 1e-12, stp = 0.01e-12, dt = .5, Ioff = -.5e-12)
    
    
Bfield.set(0e-3)
htr_setpt.set(30e-3)

ramping T8 to 50.0 mK...
50.9 m
50.7 m
50.7 m
50.9 m
51.0 m
50.5 m
50.5 m
50.3 m
50.6 m
50.2 m
50.2 m
50.2 m
49.9 m
49.6 m
49.8 m
50.1 m
50.0 m
49.6 m
49.7 m
49.5 m
49.7 m
49.6 m
T is set


Starting experimental run with id: 912


Starting experimental run with id: 913


Starting experimental run with id: 914


Starting experimental run with id: 915


...

Starting experimental run with id: 919


Starting experimental run with id: 920


Starting experimental run with id: 921


Starting experimental run with id: 922

{ 'ids' : range(912,922+1), 'B' : np.linspace(0.00e+00,1.85e-03, 11), 'T' : 0.050, 'comm : '' }
ramping T8 to 75.0 mK...


VisaIOError: ('VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.', "asking 'RDGK? 8\\n\\r' to <Model_372: LS370 input Ch>", 'getting LS370 input Ch_ch08_temperature')

In [ ]:
T_list = np.array( [200 + 25*i for i in range (5) ] )*1e-3
B_list = np.linspace(0, 1.85e-3, 11)

for jj in  jj.Tscan( T_list ):
    
    for jj in jj.Bscan(B_list):
    
        jj.IVC_fwd (amp = 2e-12, stp = 0.04e-12, dt = .5, Ioff = -.35e-12)
    
    
Bfield.set(0e-3)
htr_setpt.set(30e-3)

## Correcting the sample labeling

- The first device we measured at the begining of this cooldown is N8 instead of N1. We created db file "Experiments_59BBN1.db"

- The second device we measured was N2, which is also the one we revisiting. We saved the data in db file "Experiments_59BBNX.db" with runid 1-29. 

- This second device has non-zero I_offset and the IVc was extremely sensitive to the connection of current and voltage connection. For example in run id 14 & 19 we have the inner measuremnt panel box either open or closed. The real mechanism remained to be check.

- The third device we measured was N8, which is the last one we measured in this cooldown without the 100kohm chip resistor (they were shunted). We saved the data in db file "Experiments_59BBNX.db" with runid 29 and beyond. 

- This third one has again non-zero I_offset consistently resilient to various connections (the IVc doesn't change on the EM environment .